In [2]:
!pip3 list | grep boto3

boto3                         1.28.57


In [9]:
!aws cdk -v
"""
Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html
"""

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument command: Invalid choice, valid choices are:

accessanalyzer                           | account                                 
acm                                      | acm-pca                                 
alexaforbusiness                         | amp                                     
amplify                                  | amplifybackend                          
amplifyuibuilder                         | apigateway                              
apigatewaymanagementapi                  | apigatewayv2                         

In [19]:
import boto3
bedrock = boto3.client('bedrock')

## 使用できるモデル一覧を表示する

In [20]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'fcc267e8-94be-46a5-8b79-fd27fb5327b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 05 Nov 2023 15:12:14 GMT',
   'content-type': 'application/json',
   'content-length': '6127',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fcc267e8-94be-46a5-8b79-fd27fb5327b0'},
  'RetryAttempts': 0},
 'modelSummaries': [{'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'inputModalities': ['TEXT'],
   'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'outputModalities': ['TEXT'],
   'providerName': 'Amazon',
   'responseStreamingSupported': True},
  {'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'inputModalities': ['TEXT'],
   'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02',


## レスポンスを一括で受け取る

In [21]:
%%time

import json
bedrock_runtime = boto3.client('bedrock-runtime')
prompt = '''

Human:私は機械学習ソリューションアーキテクトに従事しておりますが、フロントエンドの開発ができるようになりたいです。ラーニングパスを教えてください。
Assistant:'''
request = {
  "modelId": "anthropic.claude-v2",
  "contentType": "application/json",
  "accept": "*/*",
  "body": json.dumps({
      'prompt':prompt,
      'max_tokens_to_sample':1000,
      'temperature':1,
      'top_k':250,
      'top_p':0.999,
      'stop_sequences':['\n\nHuman:'],
      'anthropic_version':'bedrock-2023-05-31'
  })
}

response = bedrock_runtime.invoke_model(
    **request
)
print(json.loads(response['body'].read().decode('utf-8'))['completion'])

 フロントエンド開発を学ぶには以下のような学習パスがおすすめです。

まずHTML、CSS、JavaScriptの基礎をしっかり学ぶことから始めましょう。コーディングスクールのオンラインコースや書籍、無料のオンライン教材などを利用して、HTMLとCSSでのマークアップ、JavaScriptでのインタラクティブなウェブページの作成方法を身につけていきます。

次に、JavaScriptのフレームワークを学びましょう。今最もメジャーなのはReactです。Reactの基本的な考え方と使い方を習得した上で、実践的なスキルが身につくサンプルアプリの開発などを通じて Reactの能力を高めていきます。

Frontend MastersやUdemyなどのオンラインコースもReactの学習にオススメです。

その上で、状態管理のためのRedux、CSS in JSのスタイリング手法、テストの自動化など、Reactエコシステムの主要なライブラリ・ツールの使い方を習得していきます。

最後に、TypeScriptの使用方法を学ぶことをおすすめします。静的型付けは大規模アプリの開発で重要になります。

このようにステップバイステップで学習していけば、充実したスキルを効率的に身につけることができるでしょう。プロジェクトベースの学習やオンラインコミュニティでのディスカッションも大切な学びになります。一貫した学習プランを立てて、継続的にスキルアップを図っていきましょう。
CPU times: user 27.8 ms, sys: 0 ns, total: 27.8 ms
Wall time: 25.6 s


## レスポンスを逐次受け取る（ストリームング用のAPI）

In [23]:
%%time

prompt = '''

Human:私は機械学習ソリューションアーキテクトに従事しておりますが、フロントエンドの開発ができるようになりたいです。ラーニングパスを教えてください。
Assistant:'''
request = {
  "modelId": "anthropic.claude-v2",
  "contentType": "application/json",
  "accept": "*/*",
  "body": json.dumps({
      'prompt':prompt,
      'max_tokens_to_sample':1000,
      'temperature':1,
      'top_k':250,
      'top_p':0.999,
      'stop_sequences':['\n\nHuman:'],
      'anthropic_version':'bedrock-2023-05-31'
  })
}

response = bedrock_runtime.invoke_model_with_response_stream(
    **request
)

stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            print(json.loads(chunk.get('bytes').decode()))

{'completion': ' ', 'stop_reason': None, 'stop': None}
{'completion': 'フロントエンド開発を行う', 'stop_reason': None, 'stop': None}
{'completion': 'ためには、以下のような学習', 'stop_reason': None, 'stop': None}
{'completion': 'パスが考えられます。\n\nまず', 'stop_reason': None, 'stop': None}
{'completion': 'HTML、CSS、JavaScriptの基礎をしっ', 'stop_reason': None, 'stop': None}
{'completion': 'かり身に', 'stop_reason': None, 'stop': None}
{'completion': 'つけることが', 'stop_reason': None, 'stop': None}
{'completion': '大', 'stop_reason': None, 'stop': None}
{'completion': '切です。CodecademyやUdemyなどのオンライン学', 'stop_reason': None, 'stop': None}
{'completion': '習サイトを活', 'stop_reason': None, 'stop': None}
{'completion': '用すると効', 'stop_reason': None, 'stop': None}
{'completion': '率的に学べます。\n\n次に、ReactやV', 'stop_reason': None, 'stop': None}
{'completion': 'ue.jsといったフレームワークの使', 'stop_reason': None, 'stop': None}
{'completion': 'い方を習得します。公式', 'stop_reason': None, 'stop': None}
{'completion': 'ドキュメントを読みつ', 'stop_reason': None, 'stop': None}
{'completio

## 参考

- [Amazon Bedrock で Interpreter を開発 !](https://aws.amazon.com/jp/builders-flash/202311/bedrock-interpreter/?awsf.filter-name=*all)